# Initialization

In [1]:
import logging, rx
from rx import operators
from threading import current_thread, Lock, Condition
from datetime import datetime

logger = logging.getLogger()
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.DEBUG)

def logWithTime(msg):
    logger.info(f'{datetime.now().strftime("%H:%M:%S.%f")} [{current_thread().ident:05d}] {msg}')

# Create observable functions

In [2]:
def sequenceInTime(period, elapsed, frequency):
    observable = rx.interval(period).pipe(
        operators.take_with_time(elapsed),
        operators.window(rx.interval(frequency)),
    )
    return observable

# Subscribe and operate

In [3]:
cv = Condition()
windowLock = Lock()
windowCount = 0
windows = []

def onWindow(observable):
    global windowLock
    windowItems = []
    def onItem(i):
        nonlocal windowItems
        windowLock.acquire()
        windowItems.append(i)
        windowLock.release()
    def onItemCompleted():
        global windowCount, windows
        nonlocal windowItems
        windowLock.acquire()
        logWithTime(f'window#{windowCount}: [{len(windowItems)}]{f" ({windowItems[0]} .. {windowItems[-1]})" if len(windowItems) > 0 else ""}')
        windows += windowItems
        windowCount += 1
        windowLock.release()
    observable.subscribe(
        on_next=onItem,
        on_completed=onItemCompleted,
    )

def onCompleted():
    global cv
    logWithTime(f'... stream completed')
    cv.acquire()
    cv.notify()
    cv.release()

cv.acquire()
period = .01
elapsed = 5
frequency = 1
observable = sequenceInTime(period, elapsed, frequency)
with observable.subscribe(
    on_next=onWindow,
    on_completed=onCompleted,
) as subscriber:
    startAt = datetime.now()
    logWithTime(f'start receiving from stream ...')
    if not cv.wait(elapsed + 1):
        logWithTime(f'timeout')
    logWithTime(f'total received {len(windows)} ticks')
    logWithTime(f'exit, {datetime.now() - startAt} elapsed')
cv.release()

17:31:40.369006 [04884] start receiving from stream ...
17:31:41.370494 [08016] window#0: [64] (0 .. 63)
17:31:42.375466 [10300] window#1: [60] (64 .. 123)
17:31:43.388416 [18396] window#2: [65] (124 .. 188)
17:31:44.398473 [14248] window#3: [65] (189 .. 253)
17:31:45.369490 [15016] window#4: [55] (254 .. 308)
17:31:45.372490 [15016] ... stream completed
17:31:45.374490 [04884] total received 309 ticks
17:31:45.374490 [04884] exit, 0:00:05.005484 elapsed
